In [1]:
import os

In [2]:
import base64

In [3]:
import subprocess

In [4]:
import ipywidgets as widgets
from IPython.display import HTML

In [5]:
uploader = widgets.FileUpload(
    accept=".pdf", 
    multiple=False
)

In [6]:
out = widgets.Output()

In [7]:
display(uploader, out)

FileUpload(value={}, accept='.pdf', description='Upload')

Output()

In [8]:
def run_refresh_upload(*ignore):
    input_file = "input.pdf"
    output_file = "output.pdf"
    for f in uploader.value.keys():
        file_name, file_extension = os.path.splitext(f)
        if file_extension == ".pdf":
            with open(input_file, "wb") as fp:
                fp.write(uploader.value[f]["content"])
            subprocess.check_output(["qpdf", "--decrypt", input_file, output_file])
            with open(output_file, "rb") as f:
                data = f.read()
            b64 = base64.b64encode(data)
            payload = b64.decode()
            html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
            html = html.format(payload=payload,title=title+f' {output_file}',filename=output_file)
            os.remove(input_file)
            os.remove(output_file)
            return HTML(html)
        else: 
            print('Can not convert: ', f)

In [9]:
uploader.observe(run_refresh_upload, 'value')